In [1]:
# 1) Libraries
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, TimeSeriesSplit, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from joblib import dump

In [2]:
# ----------------- Config -----------------
DATA_PATH = "household_power_consumption.txt"  # Change if needed
SEP = ";"
NA_VALUES = ["?"]
RANDOM_STATE = 42
ROLLING_WINDOW = 7   # hours
SAMPLE_FRAC = 1.0    # set <1 for faster testing
TARGET = "Global_active_power"


In [3]:
# 2) Load Data
# =====================================================
print("Loading dataset...")
data = pd.read_csv(
    DATA_PATH,
    sep=SEP,
    na_values=NA_VALUES,
    low_memory=False
)

if SAMPLE_FRAC < 1.0:
    data = data.sample(frac=SAMPLE_FRAC, random_state=RANDOM_STATE).reset_index(drop=True)
    print(f"Sampled {SAMPLE_FRAC*100:.0f}% of data.")

print("Shape:", data.shape)
print("Columns:", list(data.columns))


Loading dataset...
Shape: (2075259, 9)
Columns: ['Date', 'Time', 'Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']


In [4]:
# 3) Parse DateTime & Clean
# =====================================================
print("Parsing DateTime...")
data["DateTime"] = pd.to_datetime(
    data["Date"].astype(str) + " " + data["Time"].astype(str),
    format="%d/%m/%Y %H:%M:%S",
    errors="coerce"
)

before = len(data)
data = data.dropna(subset=["DateTime"]).copy()
print(f"Dropped {before - len(data)} invalid DateTime rows.")

# Sort for rolling features
data = data.sort_values("DateTime").reset_index(drop=True)

# Convert measurement columns to numeric
num_cols_raw = [
    "Global_active_power", "Global_reactive_power", "Voltage",
    "Global_intensity", "Sub_metering_1", "Sub_metering_2", "Sub_metering_3"
]
for col in num_cols_raw:
    data[col] = pd.to_numeric(data[col], errors="coerce")

# Replace negatives with NaN
for col in num_cols_raw:
    neg_ct = (data[col] < 0).sum(skipna=True)
    if neg_ct > 0:
        print(f"{col}: found {neg_ct} negatives → set to NaN.")
        data.loc[data[col] < 0, col] = np.nan

print("Missing counts before imputation:\n", data[num_cols_raw].isna().sum())

# Drop original Date/Time strings
data.drop(columns=["Date", "Time"], inplace=True, errors="ignore")


Parsing DateTime...
Dropped 0 invalid DateTime rows.
Missing counts before imputation:
 Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64


In [5]:
# 4) Feature Engineering
# =====================================================
print("Engineering features...")

# Date-related
data["Date_only"] = data["DateTime"].dt.date
data["Hour"] = data["DateTime"].dt.hour
data["Month"] = data["DateTime"].dt.month
data["Weekday"] = data["DateTime"].dt.weekday
data["Is_Weekend"] = data["Weekday"].isin([5, 6]).astype(int)

# Daily average
daily_avg = (
    data.groupby("Date_only")["Global_active_power"]
        .mean()
        .reset_index(name="Daily_Avg_Power")
)
data = data.merge(daily_avg, on="Date_only", how="left")

# Peak hour fix — skip all-NaN days
valid_days = data.groupby("Date_only")["Global_active_power"].transform(lambda x: x.notna().any())
data_valid_peak = data[valid_days]
idxmax_per_day = data_valid_peak.groupby("Date_only")["Global_active_power"].idxmax()
idxmax_per_day = idxmax_per_day.dropna().astype(int)
peak_hours = (
    data.loc[idxmax_per_day, ["Date_only", "Hour"]]
        .rename(columns={"Hour": "Peak_Hour"})
)
data = data.merge(peak_hours, on="Date_only", how="left")

# Rolling average
data["Rolling_Avg_Power"] = (
    data["Global_active_power"]
    .rolling(window=ROLLING_WINDOW, min_periods=1)
    .mean()
)

# Fill missing target before splitting
target_median = data[TARGET].median()
data[TARGET] = data[TARGET].fillna(target_median)


Engineering features...


In [6]:
# 5) Train/Test Split (Chronological)
# =====================================================
print("Splitting train/test...")
exclude_cols = {"DateTime", "Date_only", TARGET}
feature_cols = [c for c in data.columns if c not in exclude_cols]

n = len(data)
split_idx = int(n * 0.8)
train = data.iloc[:split_idx].copy()
test = data.iloc[split_idx:].copy()

X_train, y_train = train[feature_cols], train[TARGET]
X_test, y_test = test[feature_cols], test[TARGET]

print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")


Splitting train/test...
Train size: (1660207, 13), Test size: (415052, 13)


In [7]:
# 6) Preprocessors
# =====================================================
numeric_features = X_train.columns.tolist()

preprocess_scaled = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
preprocess_tree = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])


In [8]:
# 7) Models
# =====================================================
models = {
    "Linear Regression": Pipeline([
        ("prep", preprocess_scaled),
        ("model", LinearRegression())
    ]),
    "Random Forest": Pipeline([
        ("prep", preprocess_tree),
        ("model", RandomForestRegressor(
            n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1
        ))
    ]),
    "Gradient Boosting": Pipeline([
        ("prep", preprocess_tree),
        ("model", GradientBoostingRegressor(
            n_estimators=200, random_state=RANDOM_STATE
        ))
    ]),
    "Neural Network": Pipeline([
        ("prep", preprocess_scaled),
        ("model", MLPRegressor(
            hidden_layer_sizes=(64, 32),
            activation="relu",
            solver="adam",
            learning_rate_init=1e-3,
            max_iter=50,
            random_state=RANDOM_STATE
        ))
    ])
}


In [ ]:
# 8) Train & Evaluate
# =====================================================
def eval_regression(y_true, y_pred, label=""):
    rmse = root_mean_squared_error(y_true, y_pred)  # updated to avoid warning
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"\n[{label}] RMSE: {rmse:.4f} | MAE: {mae:.4f} | R²: {r2:.4f}")
    return {"model": label, "RMSE": rmse, "MAE": mae, "R2": r2}

results = []
fitted_models = {}

print("Training models...")
for name, pipe in models.items():
    print(f"→ {name}")
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    results.append(eval_regression(y_test, preds, label=name))
    fitted_models[name] = pipe

results_df = pd.DataFrame(results).sort_values("RMSE")
print("\nModel Comparison:\n", results_df)



Training models...
→ Linear Regression

[Linear Regression] RMSE: 0.0387 | MAE: 0.0235 | R²: 0.9981
→ Random Forest


In [ ]:
# 9) Hyperparameter Tuning (Random Forest)
# =====================================================
print("\nTuning Random Forest...")
tscv = TimeSeriesSplit(n_splits=3)
rf_pipe = models["Random Forest"]

param_dist = {
    "model__n_estimators": [150, 200, 300],
    "model__max_depth": [None, 10, 20, 40],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4],
    "model__max_features": ["sqrt", "log2", None]
}

rf_search = RandomizedSearchCV(
    rf_pipe,
    param_distributions=param_dist,
    n_iter=8,
    scoring="neg_root_mean_squared_error",
    cv=tscv,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=1
)
rf_search.fit(X_train, y_train)

print("Best RF Params:", rf_search.best_params_)
rf_best = rf_search.best_estimator_
results.append(eval_regression(y_test, rf_best.predict(X_test), "Random Forest (tuned)"))


In [ ]:
# 10) Save Best Model
# =====================================================
results_df = pd.DataFrame(results).sort_values("RMSE")
best_name = results_df.iloc[0]["model"]
best_model = rf_best if "tuned" in best_name else fitted_models[best_name]

os.makedirs("models", exist_ok=True)
model_path = f"models/best_model_{best_name.replace(' ', '_')}.joblib"
dump(best_model, model_path)
print(f"\n✅ Saved best model: {best_name} → {model_path}")


In [ ]:
# 11) Preview Predictions
# =====================================================
preview = pd.DataFrame({
    "y_true": y_test.reset_index(drop=True)[:10],
    "y_pred": pd.Series(best_model.predict(X_test)[:10])
})
print("\nPrediction Preview:\n", preview)
